In [150]:
import pandas as pd
import numpy as np
from sklearn import linear_model, metrics
from sklearn.model_selection import train_test_split

* Importar as bases de treino e teste para o desafio

In [216]:
data= pd.read_csv("train.csv")
data2= pd.read_csv("test.csv")

* separar a base em targuet e explicativas 

In [217]:
targuet = data["NU_NOTA_MT"]

In [218]:
# Esta parte foi colocada aqui, pois na hora de executar o modelo na base de teste as colunas não eram as mesmas da base de treino e isso dava erro
data = data[data2.columns]

In [219]:
data.shape
#(13730, 47)

(13730, 47)

In [229]:
data2.shape

(4576, 47)

<h1>Vamos escolher e tratas as features da base de treino para criar nosso modelo</h1>

In [220]:
porcentagem_nulos = pd.DataFrame(data.isnull().sum()/data.shape[0]*100,columns=['porcentagem'])

* Para tentar melhorar o modelo vamos remover as features com mais de 50% de nulos

In [221]:
porcentagem_nulos[porcentagem_nulos.porcentagem > 50].T

,TP_ENSINO,TP_DEPENDENCIA_ADM_ESC,Q027
porcentagem,68.812819,68.812819,53.699927


* baseado no Dicionario_Microdados_Enem_2016 e nas features disponiveis na base de testes vamos listar as features para gerar Dummies depois

In [222]:
fazer_dummies = [
 'CO_UF_RESIDENCIA',
 'SG_UF_RESIDENCIA',
 'TP_SEXO',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 #'TP_ENSINO', #mais que 50% nulo
 'IN_TREINEIRO',
 #'TP_DEPENDENCIA_ADM_ESC', #mais que 50% nulo
 'IN_BAIXA_VISAO',
 'IN_CEGUEIRA',
 'IN_SURDEZ',
 'IN_DISLEXIA',
 'IN_DISCALCULIA',
 'IN_SABATISTA',
 'IN_GESTANTE',
 'IN_IDOSO',
 'TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'TP_LINGUA',
 'TP_STATUS_REDACAO',
 'Q001',
 'Q002',
 'Q006',
 'Q024',
 'Q025',
 'Q026',
 #'Q027', #mais que 50% nulo
 'Q047']

In [230]:
# criar as dimmies e atribuir na base de dados
dummies = pd.get_dummies(data[fazer_dummies])
data[dummies.columns] = dummies

dummies = pd.get_dummies(data2[fazer_dummies])
data2[dummies.columns] = dummies

In [231]:
data.shape

(13730, 124)

In [232]:
data2.shape

(4576, 124)

* Para fazer a analise, não vamos usar todas as features e temos que retirar as features que deram origem as dummies

In [225]:
# 
colunas_para_remover = ['SG_UF_RESIDENCIA',
 'CO_UF_RESIDENCIA',
 'NU_INSCRICAO',
 'TP_SEXO',
 'TP_COR_RACA',
 'TP_NACIONALIDADE',
 'TP_ST_CONCLUSAO',
 'TP_ANO_CONCLUIU',
 'TP_ESCOLA',
 'TP_ENSINO',
 'IN_TREINEIRO',
 'TP_DEPENDENCIA_ADM_ESC',
 'IN_BAIXA_VISAO',
 'IN_CEGUEIRA',
 'IN_SURDEZ',
 'IN_DISLEXIA',
 'IN_DISCALCULIA',
 'IN_SABATISTA',
 'IN_GESTANTE',
 'IN_IDOSO',
 'TP_PRESENCA_CN',
 'TP_PRESENCA_CH',
 'TP_PRESENCA_LC',
 'CO_PROVA_CN',
 'CO_PROVA_CH',
 'CO_PROVA_LC',
 'CO_PROVA_MT',
 'TP_LINGUA',
 'TP_STATUS_REDACAO',
 'Q001',
 'Q002',
 'Q006',
 'Q024',
 'Q025',
 'Q026',
 'Q027',
 'Q047']


In [226]:
explanatory_features = list(data.columns)
for i in colunas_para_remover:
    explanatory_features.remove(i)
explanatory_features = data[explanatory_features]
#(13730, 78)

In [227]:
explanatory_features.shape

(13730, 87)

In [181]:
explanatory_features.dtypes

NU_IDADE           int64
NU_NOTA_CN       float64
NU_NOTA_CH       float64
NU_NOTA_LC       float64
NU_NOTA_COMP1    float64
                  ...   
Q047_A             uint8
Q047_B             uint8
Q047_C             uint8
Q047_D             uint8
Q047_E             uint8
Length: 87, dtype: object

* para melhorar o modelo vamos converter as notas de float para int e assumir que os nulos são nota zero

In [184]:
teste = pd.DataFrame()
for i in explanatory_features.columns:
    if explanatory_features[i].dtype == 'float':
        teste[i]=(explanatory_features[i].fillna(0)*100).astype(int)
teste.describe()
explanatory_features[teste.columns] = teste[teste.columns] 

In [196]:
#verificar se estão nos tipos corretos
teste.dtypes

NU_NOTA_CN         int32
NU_NOTA_CH         int32
NU_NOTA_LC         int32
NU_NOTA_COMP1      int32
NU_NOTA_COMP2      int32
NU_NOTA_COMP3      int32
NU_NOTA_COMP4      int32
NU_NOTA_COMP5      int32
NU_NOTA_REDACAO    int32
dtype: object

* para melhorar o modelo vamos converter as notas de float para int e assumir que os nulos são nota zero e vamos salvar em uma variavel diferente para não modificar a targuet original, para não perder os valores apos a virgula, vamos multiplicar por 100

In [197]:
alvo = (targuet.fillna(0)*100).astype(int)

* Dividir a base em treino e teste

In [198]:
x_train, x_test, y_train, y_test = train_test_split(explanatory_features, alvo, test_size=0.33, random_state=42)

<h1>Vamos rodar o modelo normalizando e não normalizando o modelo</h1>

In [199]:
# Create linear regression object
regr = linear_model.LinearRegression(normalize=False,n_jobs=4)

# Train the model using the training sets
regr.fit(x_train, y_train)

# Create linear regression object
regr2 = linear_model.LinearRegression(normalize=True,n_jobs=4)

# Train the model using the training sets
regr2.fit(x_train, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=4, normalize=True)

* Apos treinar o modelo, vamos voltar as notas para a ordem de grandeza corretas

In [200]:
y_test=y_test/100

* Vamos fazer as predições usando os modelos treinados

In [201]:
# Make predictions using the testing set
#nao_normalizado
y_test_pred = regr.predict(x_test)
y_test_pred = y_test_pred/100

#normalizado
y_test_pred2 = regr2.predict(x_test)
y_test_pred2 = y_test_pred2/100


In [202]:
y_test_pred2.min()

-25.6

* como as notas não podem ser negativas, então vamos substituir por zero

In [203]:
#nao_norm_zera_neg
y_test_pred_1 = y_test_pred.copy()
y_test_pred_1[y_test_pred_1<0] = 0

#norm_zera_neg
y_test_pred2_1 = y_test_pred2.copy()
y_test_pred2_1[y_test_pred2_1<0] = 0

* outra opção é converter para positivo

In [204]:
#nao_norm_abs
y_test_pred_2 = y_test_pred.copy()
y_test_pred_2[y_test_pred_2<0] = y_test_pred_2[y_test_pred_2<0]*-1

#norm_abs
y_test_pred2_2 = y_test_pred2.copy()
y_test_pred2_2[y_test_pred2_2<0] = y_test_pred2_2[y_test_pred2_2<0]*-1

* Vamos ver as metricas das tentativas

In [205]:
metricas= pd.DataFrame(columns=['Mean Absolute Error:','Mean Squared Error:','Root Mean Squared Error:'],
index=['nao_normalizado','normalizado','nao_norm_zera_neg','norm_zera_neg','nao_norm_abs','norm_abs'],
data=[
[metrics.mean_absolute_error(y_test, y_test_pred),metrics.mean_squared_error(y_test, y_test_pred),np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))],
[metrics.mean_absolute_error(y_test, y_test_pred2),metrics.mean_squared_error(y_test, y_test_pred2),np.sqrt(metrics.mean_squared_error(y_test, y_test_pred2))],
[metrics.mean_absolute_error(y_test, y_test_pred_1),metrics.mean_squared_error(y_test, y_test_pred_1),np.sqrt(metrics.mean_squared_error(y_test, y_test_pred_1))],
[metrics.mean_absolute_error(y_test, y_test_pred2_1),metrics.mean_squared_error(y_test, y_test_pred2_1),np.sqrt(metrics.mean_squared_error(y_test, y_test_pred2_1))],
[metrics.mean_absolute_error(y_test, y_test_pred_2),metrics.mean_squared_error(y_test, y_test_pred_2),np.sqrt(metrics.mean_squared_error(y_test, y_test_pred_2))],
[metrics.mean_absolute_error(y_test, y_test_pred2_2),metrics.mean_squared_error(y_test, y_test_pred2_2),np.sqrt(metrics.mean_squared_error(y_test, y_test_pred2_2))]])

# nao_normalizado	50.164493	4743.872600	68.875777
# normalizado	50.165393	4743.980423	68.876559
# nao_norm_zera_neg	48.371068	4718.357238	68.690299
# norm_zera_neg	48.368426	4718.419648	68.690754
# nao_norm_abs	50.164493	4743.872600	68.875777
# norm_abs	50.165393	4743.980423	68.876559
# nao_norm_abs_max_lim	48.371068	4718.357238	68.690299
# norm_abs_max_lim	48.368426	4718.419648	68.690754

In [206]:
metricas

,Mean Absolute Error:,Mean Squared Error:,Root Mean Squared Error:
nao_normalizado,50.158176,4743.828278,68.875455
normalizado,49.877515,4746.363388,68.893856
nao_norm_zera_neg,48.388063,4718.893157,68.694200
norm_zera_neg,48.500759,4729.625926,68.772276
nao_norm_abs,50.158176,4743.828278,68.875455
norm_abs,49.877515,4746.363388,68.893856


* O melhor modelo foi o modelo em que substituimos as predições negativas por zero

<h1>Vamos aplicar o modelo na base de testes</h1>

* Vamos selecionar as features da mesma forma ue foi feita na base de treino

In [233]:
explanatory_features = list(data2.columns)
for i in colunas_para_remover:
    explanatory_features.remove(i)
explanatory_features = data2[explanatory_features]
explanatory_features.shape

(4576, 87)

In [234]:
teste = pd.DataFrame()
for i in explanatory_features.columns:
    if explanatory_features[i].dtype == 'float':
        teste[i]=(explanatory_features[i].fillna(0)*100).astype(int)
teste.describe()
explanatory_features[teste.columns] = teste[teste.columns] 

* Vamos criar o Data Frame de resposta

In [236]:
answer = pd.DataFrame()
answer['NU_INSCRICAO'] = data2['NU_INSCRICAO']
answer['NU_NOTA_MT']= regr2.predict(explanatory_features)

In [237]:
#norm_zera_neg
y_test_pred2_1 = answer['NU_NOTA_MT'].copy()
y_test_pred2_1[y_test_pred2_1<0] = 0
answer['NU_NOTA_MT'] = y_test_pred2_1/100

<h1>Vamos salvar as respostas como 'answer.csv'</h1>

In [239]:
answer.to_csv('answer.csv',index=0)